## Importing libraries

In [ ]:
# libraries
from __future__ import print_function, division
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader

import BayesianLayers
from compression import compute_compression_rate, compute_reduced_weights
from utils import visualize_pixel_importance, generate_gif, visualise_weights

import os, sys
import itertools
from scipy import linalg
import matplotlib as mpl
!pip install pyomo
!pip install pypsa
!pip install pandapower
import pypsa
import pandapower as pp
import pandapower.networks as pn
from pandapower.estimation import estimate
import random as rand
import pandas as pd
from sklearn import mixture
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
sns.set()
rand.seed(2020)
tf.random.set_seed(2020)
torch.manual_seed(2020)

In [ ]:
from data_preprocessing import extract_consumption_values
from bic import bic_criterion_gmm
from define_network import define_network
from generate_samples import generate_samples
from compress_model import compress_model

## Defining parameters

### Obs: case_14s and case_57 correspond to the IEEE standard test cases. case_3 corresponds to the 3-bus minimal example network on PyPSA, while opf_storage_hvdc corresponds to the PyPSA opf_storage_hvdc example.

In [ ]:
net_case = "case_3" # Select ['case_14s', 'case_57', 'case_3', 'opf-storage-hvdc']
N = 60000.  # number of data points in the training set
n_of_network_samples = 100
n_of_gaussians = 5
train_frac = 0.8
percent_of_measurements = 0.4
estimation_method = 'bad_data_removal' # Select ['standard','bad_data_removal']
layer_proportions = [20,10]
net_type = 'case_3' # Select ['pp','case_3', 'opf-storage-hvdc'] pp is for pandapower cases 14s and 57

In [ ]:
os.makedirs("./plots/", exist_ok=True)

In [ ]:
os.system("unzip opf-storage-hvdc.zip")

## Fitting Solar and Normal data separately with Gaussian Mixture
#### Initially, a GMM with 3 components will be fitted, like in the original paper. Later, other pdfs will be tested.

In [ ]:
os.system("unzip electricity_data.zip")
base_path = "./electricity_data/"

train_data_path = base_path + "2011-2012 Solar home electricity data v2.csv"
test_data_path = base_path + "2012-2013 Solar home electricity data v2.csv"

In [ ]:
train_data = pd.read_csv(train_data_path, skiprows=1)
test_data = pd.read_csv(test_data_path, skiprows=1)

### GC = General Consumption for electricity supplied all the time (primary tariff, either inclining block or time of use rates), excluding solar generation and controlled load supply 
### CL = Controlled Load Consumption (Off peak 1 or 2 tariffs)
### GG = Gross Generation for electricity generated by the solar system with a gross metering configuration, measured separately to household loads

In [ ]:
test_data.tail()

In [ ]:
train_data_general, train_data_solar = extract_consumption_values(train_data)
test_data_general, test_data_solar = extract_consumption_values(test_data)

In [ ]:
train_values = np.reshape(train_data_general.values, (train_data_general.shape[0]*train_data_general.shape[1],1))
test_values = np.reshape(test_data_general.values, (test_data_general.shape[0]*test_data_general.shape[1],1))

In [ ]:
bic_criterion_gmm(10, train_values, test_values)

#### n_of_components=5 was initially chosen

In [ ]:
gmm = mixture.GaussianMixture(n_components=n_of_gaussians)
#fit it to the data
gmm.fit(train_values)

#### Solar consumption data was either 0 or NaN. We will only fit GMM on the General Consumption values.

## Generating samples from Network

In [ ]:
net = define_network(net_case)

In [ ]:
injection_values, network_state_samples, measurement_vector = generate_samples(net_case,n_of_network_samples,net, percent_of_measurements, estimation_method, gmm, net_type)

## Fitting NN Model

In [ ]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

model = tf.keras.models.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(layer_proportions[0]*measurement_vector.shape[1],input_dim=measurement_vector.shape[1]),
    tf.keras.layers.Dense(layer_proportions[1]*measurement_vector.shape[1], input_dim=layer_proportions[0]*measurement_vector.shape[1]),
    tf.keras.layers.Dense(network_state_samples.shape[1], input_dim=layer_proportions[1]*measurement_vector.shape[1]),
])

model.compile(
    loss='mse', # keras.losses.mean_squared_error
    optimizer=tf.keras.optimizers.SGD(lr=0.2),
)

print("Training ... With default parameters, this takes less than 10 seconds.")
training_history = model.fit(
    injection_values[:int(measurement_vector.shape[0]*train_frac)], # input
    network_state_samples[:int(measurement_vector.shape[0]*train_frac)], # output
    batch_size=int(measurement_vector.shape[0]*train_frac),
    verbose=0, # Suppress chatty output; use Tensorboard instead
    epochs=50,
    validation_data=(injection_values[int(measurement_vector.shape[0]*train_frac):], network_state_samples[int(measurement_vector.shape[0]*train_frac):]),
    callbacks=[tensorboard_callback],
)

print("Average test loss: ", np.average(training_history.history['loss']))


In [ ]:
plt.plot(np.log(training_history.history['loss']))

In [ ]:
plt.scatter(model.predict(injection_values[-10:]),network_state_samples[-10:])

### The cell below runs the Bayesian Compression step. The variable "Test loss" is the MSE reported in the draft, while the compression factor is the one based on the weight uncertainty.

In [ ]:
if __name__ == '__main__':

    class FLAGS:
        epochs=50
        batchsize=100
        thresholds=[-2.8, -3., -5.]
        
    FLAGS.cuda = torch.cuda.is_available()  # check if we can put the net on the GPU

    compress_model(injection_values, measurement_vector, train_frac, network_state_samples, layer_proportions, N, FLAGS)